# Treatment of Covid-19 with Remdesivir - Systematic Review and Meta-Analysis
In this notebook you will be guided through all the steps necessary to conduct this systematic review. We have automated as many steps as possible, but there are still a few steps you need to perform manually. If you are unfamiliar with jupyter notebooks, the way to run code within a cell is by clicking on the cell and clicking on the play button or pressing shift+enter. Please be careful to precisely do each step as is explained here, or your results might not work properly.
<br>first let's import all necessary python packages.

In [1]:
from bs4 import BeautifulSoup
import copy
import forestplot as fp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import re
from urllib.request import Request, urlopen
from urllib.error import HTTPError

## Search PubMed for appropriate studies
PubMed is a database which hosts over 35 million scientific publications in the biomedical literature. Most studies on covid are also available free of charge on their website. The first step is to search for relevant publications. We are interested in studies about covid-19 and remdesivir. More specifically Remdesivir must be mentioned in the title of the publication and either covid, corona, or sars-cov-2 must be mentioned in the title or abstract. We are only looking for original studies and not for other meta-analyses on the subject, so we exclude publications which have the term meta-analysis in the title. Please follow the instructions carefully.
1. Click on this [link](https://pubmed.ncbi.nlm.nih.gov/) to visit PubMed.
2. Copy this prompt `(((((((covid[Title/Abstract]) OR (corona[Title/Abstract])) OR (sars-cov-2[Title/Abstract])) OR (sars cov 2[Title/Abstract])) AND (remdesivir[Title])) NOT (Meta Analysis[Title])) NOT (Meta-Analysis[Title])) NOT (Review[Title])` into the search box.
3. Click on save and select <b>all results</b> (not all resutls on page) in selection and <b>PubMed</b> in format
4. Click on create file and save it under the name <b>search_results.txt</b>

## Automated Filtering
### Filter for Randomized Controlled Trials
We only want to include randomized control trials in our meta-analysis. We are using a machine-learning tool called robotsearch to filter out only the studies from the PubMed search which used a randomized control trial design. If you followed the README.md precisely, you already installed the tool. Please follow the instructions carefully.
1. Move the file search_results.txt that you just downloaded into the robotsearch directory, which is found within the overall project-directory.
2. Open a **new anaconda terminal**. Don't close the one running!
3. `cd` your way into the robotsearch directory within the overall covid_systematic_review directory you downloaded from GitHub. To learn about how to change directories in a terminal [visit here](https://www.lifewire.com/change-directories-in-command-prompt-5185508)
4. If the environment is not activated type `conda activate covid_review`. You will know if your environment is activated by the first phrase in the parentheses. It should say (covid_review).
5. Conda should now be displaing (covid_review) ...\robotsearch>. Run `robotsearch search_results.txt -p` Don't forget the -p-flag, or the results will contain too many false positives you will have to manually check later.

The results are saved in the file search_reults_robotviewer_RCTs.txt. Let's look at the result, by simply running the cell below.

In [2]:
from robotsearch.parsers import ris
file_input = "robotsearch/search_results.txt"
file_result = "robotsearch/search_results_robotreviewer_RCTs.txt"
with open(file_input, 'r', encoding="utf8") as f:
    inp = ris.load(f)
with open(file_result, 'r', encoding="utf8") as f:
    result = ris.load(f)
print("The inital search result has {} articles".format(len(inp)))
print("{} articles were classified as rcts".format(len(result)))

other non numbered
other non numbered
The inital search result has 821 articles
30 articles were classified as rcts


Now we know how many articles were classified as randomized-controlled-trials. We are only going to use the ones that have an open-access. Those that do, have a pmc- identifier associated with the publication. Please execute the cell below to filter out the pmcs.

In [3]:
#extract PMC - they are found in the 'PMC' key.
pmcs = []
for i in range(len(result)):
    if "PMC" in result[i].keys():
        id_raw = result[i]['PMC'][0]
        pmc = id_raw.strip()
        pmcs.append(pmc)

print("{} studies have a pmc-identifier and can be accessed through pubmed open-access".format(len(pmcs)))

26 studies have a pmc-identifier and can be accessed through pubmed open-access


### Filter for response-measure of interest
Let's reduce the number of publications even further by checking if the articles contain appropriate response-measures. The response-measures we are interested in are indicators for mortality or clinical recovery or improvement. The statistical measures of interest are hazard-ratio or rate-ratio. Every study reports those in the methods section or later in the article. The following cell scans the open-access texts on PubMed and checks if either the terms "mortality" or death or the terms "time to clinical improvement/recovery" are mentioned after the key-word "method". And the check if at the same time the keywords "hazard/rate-ratio" are mentioned after the "method" keyword. Please execute the cell below. It will take a few minutes to run. If you run into errors, try installing Mozilla Firefox if you don't have it on your system.

In [ ]:
site= "https://www.ncbi.nlm.nih.gov/pmc/articles/{}/"
hdr = {'User-Agent': 'Mozilla/5.0'}
regex_mortality = r"method.+(?:(?:mortality|death)|(?:time to(?:clinical improvement|recovery)))" # the outcome meassure must come after the word method, so it is likey mentioned in the methods or results section
regex_hr = r"method.+(?:hazard|rate)[\s-]ratio"
pmcs_outcome = []
for i in range(len(pmcs)):
    print(i)
    url = site.format(pmcs[i])
    req = Request(url,headers=hdr)
    try:
        page = urlopen(req)
        soup = BeautifulSoup(page)
        if re.search(regex_mortality, soup.prettify(), re.DOTALL|re.IGNORECASE) and re.search(regex_hr, soup.prettify(), re.DOTALL|re.IGNORECASE):
            print(pmcs[i], "has a propper response-measure")
            pmcs_outcome.append(pmcs[i])
    except HTTPError:
        print("httperror")
print("{} articles contain mortality/death or time to clinical improvement or recovery and propper outcome meassure".format(len(pmcs_outcome)))
with open("papers.txt", "w") as f:
    for pmc in pmcs_outcome:
        f.write("%s\n" % "https://www.ncbi.nlm.nih.gov/pmc/articles/{}/".format(pmc))

### Create a spreadsheet to record screening results.
Now that we reduced the number of eligible studies by a very significant amount, we will inspect the remaining studies manually. Run the cell below to create a spreadsheet in which you will document the results of your inspection. You will also get a list of links where you will find the studies.

In [ ]:
screening_results = pd.DataFrame(columns = ['pmc', 'non_retracted', 'randomized_controlled', 'adults', 'infected', 'remdesivir_vs_control', 'propper_outcome'])
screening_results['pmc'] = np.array(pmcs_outcome)
screening_results.to_csv("screening_results.csv", index = False, sep = ";")
for i in range(len(pmcs_outcome)):
    print("https://www.ncbi.nlm.nih.gov/pmc/articles/{}/".format(pmcs_outcome[i]))

## Manual Screening of Articles
Now that we filtered out the promising studies, we must manually check for their eligibility. Even though we automated the classification as randomized control trial, we must check for errors. We follow the following protocol. There are certain criteria a publication has to meet in order to be eligible for our final analysis. Open each of the HTMLs above by clicking on the link or open the file papers.txt. Apply the protocol below very carefully to each study individually. Open the file **screening_results.csv** in a csv-editor of your choice. You will find the file in the main project directory. Note that the separator is a semicolon. <br>
For each criterion listed below, the spreadsheet has a column in which to document, if the study meets the criterion. Type **1** into the cell if the **criterion is met** and **0** if it is **not met**. You can stop checking the remaining criteria for that particular paper, once you coded one criterion as 0 and continue to the next paper, by clicking on the next link in the list above.
### Not retracted
Rarely it happens, that already published articles are retracted for various reasons. This probably won't be the case for the studies here, but we still must check. If the article has been retracted, it shows in a red box on top of the webpage. It's impossible to miss. Code **1** if the article has **not been retracted**.
### Randomized Controlled Trial
Randomized Clinical Trials are those in which patients are assigned randomly to either a group receiving treatment with Remdesivir or a control group, which may be a placebo-group or a standard-treatment. Check the abstract and/or methods section to check if the studies are randomized control trials. Don't worry about missing this information, the authors will make it very clear if their study is a RCT. Keywords indicating that a study is not an RCT are: "retrospective" study or analysis or "observational cohort" study.
### Majority Adults. 
Check if proportion of age above 18 is >95%.
### Participants infected 
Participants must be infected with Covid-19. **No** studies analyzing remdesivir as a **prevention** treatment should be included.
### Remdesivir vs Control 
The only difference between treatment- and control-group should be whether patients received remdesivir or not. Some studies investigate the effect of combination treatments of remdesivir with other medications - don't include those studies. Control groups can be placebo or standard-care. If multiple treatments are tested, the study must include a comparison of remdesivir with a placebo or standard care group. **Don't include studies**, in which **both** experimental and control group receive remdesivir but only different dosages.
### Propper outcome
To assess the effectiveness of treatment with remdesivir compared to the placebo or standard-care, there must be a proper effect-measure reported. 
Check if **at least one** of the outcomes **mortality, time to recovery or time to clinical improvement or equivalent** measures has one of these measures associated with it: **hazard-ratio or rate-ratio**. Keep in mind that what the authors declare as primary outcome might not be the primary outcome for this study. You might even have to look into the appendices to retrieve the outcome measure.

## Extract eligible studies
Make sure to **save the file**. We are going to use the studies that fulfill the above criteria. Execute the cell below to get the studies to be included in the final analysis.


In [ ]:
# Please make sure to save your screening_results.csv properly. In Germany the default seperator is semicolon.
df_screening = pd.read_csv("screening_results.csv",sep=";")
df_screening = df_screening.fillna(0)
df_screening["eligable"] = (df_screening.iloc[:,1:].sum(axis = 1)/6).astype(int).astype(bool)
eligable_studies = list(df_screening.loc[df_screening.eligable == True,"pmc"])
result_table = pd.DataFrame(columns = ['pmc', 'ratio_mortality', 'lower_confidence_bound_mortality', 'upper_confidence_bound_mortality', 'p_value_mortality', 'sample_size_remdesivir_mortality', 'sample_size_control_mortality', 'ratio_recovery', 'lower_confidence_bound_recovery', 'upper_confidence_bound_recovery', 'p_value_recovery','sample_size_remdesivir_recovery', 'sample_size_control_recovery', 'placebo_controlled'])
result_table['pmc'] = np.array(eligable_studies)
result_table.to_csv("result_table.csv", index = False, sep = ";")
print("{} studies were found to be eligable".format(len(eligable_studies)))
for i in range(len(eligable_studies)):
    print("https://www.ncbi.nlm.nih.gov/pmc/articles/{}/".format(eligable_studies[i]))

## Extract Relevant Data
Good job thus far. We have found all the studies which will be included in the final analysis. If you discover that a criterion hasn't been met after all, simply change the sceening_results.csv file accordingly, save it, and and rerun the cell above. <br>
Open the file result-table.csv. If you use Microsoft Excel, some fields might be automatically transformed into Dates ,while entering your results, so pay attention. Even if you live in Germany and your spreadsheet programm expects comma as decimal-separator, always use the "." as decimal-point. The csv-separator is semicolon. Open the studies one-by-on by clicking on the corresponding URL in the output of the cell above. Please follow the protocol below when you record your findings. If a measure is missing, leave the cell blank.
### ratio_mortality
Type the value of the hazard- or rate-ratio for mortality. You might have to look into an appendix mentioned in the paper. If mortality-ratios are reported for different time periods, choose the latest. If a study includes hospitalization or death don't include this study for the mortality-measure, because death itself is the outcome measure. If there are ratios reported for different subgroups, use the main ratio, comparing the entire remdesivir-group to the entire controll-group.
### lower_confidence_bound_mortality
The confidence-interval is usually reported after the ratio. E.g. [95% CI 0·81–1·61] - the example is from an actual paper. You can see that the decimal point is missing. Please type the lower confidence bound (the left one) into the result_table. In the example it would be 0.81.
### upper_confidence bound mortality
Equivalently type the upper confidence bound (the right one) into the table. In the example it would be 1.61.
### p_value_mortality
Type the p-value into the result-table. It is usually documented right next to the confidence interval. If it isn't you have to look a bit in the results section. If there is only a confidence interval reported, leave the column p-value empty.
## sample-sizes
Type the sample-sizes for the remdesivir group and the control-group separately. Make sure to check for the correct sample sizes for the mortality-ratio. Sometimes sample-sizes for different measures differ, because measures are not recorded for all patients.
### ratio_recovery
Different studies use different recovery-measures. You should only report one. If a ratio is reported for time to recovery use that one, if not, look if there is time to improvement or alleviation of symptoms or an equivalent measure. The indicator for aleviation of symptoms might require a symptom-check-questionnaire to compare the treatment effect to. Only report this meassure, if the questionnaire was filled out prior to beginning of treatment.
Type the value of the hazard- or rate-ratio into the cell. You might have to look into an appendix mentioned in the paper.
### lower_confidence_bound_recovery
The confidence-interval is usually reported after the ratio. E.g. [95% CI 0·81–1·61] - the example is from an actual paper. You can see that the decimal point is missing. Please type the lower confidence bound (the left one) into the result_table. In the example it would be 0.81.
### upper_confidence_bound_recovery
Equivalently type the upper confidence bound (the right one) into the table. In the example it would be 1.61.
### p_value_recovery
Type the p-value into the result-table. It is usually documented right next to the confidence interval. If it isn't you have to look a bit in the results section. If there is only a confidence interval reported, leave the column p-value empty.
### sample-sizes
Type the sample-sizes for the remdesivir group and the control-group separately. Make sure to check for the correct sample sizes for the recovery-ratio. Sometimes sample-sizes for different measures differ, because measures are not recorded for all patients.
### placebo
check if the control group received a placebo or not. If the control-group received a placebo type 1 into the column and else type 0.
### save the file.


## Data Analysis
The analysis is almost done. The manual work is over. It's time to look at our results. We have two effects we want to evaluate - Does remdesivir reduce mortality and does remdesivir reduce the time to recovery or clinical improvement. Run the cells below to get the results.
### Preprocess Data
This function calculates the pooled effect for multiple studies. It utilizes the fixed-effects model.

In [2]:
def pooled_effect(df: pd.DataFrame)->tuple:
    """takes in a data-frame and calculate the pooled fixed effect-size for a meta-analysis"""
    
    df_effect = copy.copy(df)
    # choose the columns with hazard/rate-ratio, lower confidence-bound and upper confidence-bound
    if "ratio_mortality" in df_effect.columns:
        df_effect = df_effect[["ratio_mortality", "lower_confidence_bound_mortality", "upper_confidence_bound_mortality"]]
    else:
        df_effect = df_effect[["ratio_recovery", "lower_confidence_bound_recovery", "upper_confidence_bound_recovery"]]
    # rename the columns to unify both types of outcomes
    df_effect.columns = ["ratio", "l", "u"]
    
    # the confidence intervals can be used to estimate the standard-errors. The log of the ratios is usually apr. normally distributed.
    # The critical value for a two-sided 95% confidence interval is 1.96. We calculate the difference of the ci-bounds and divide
    # by 2 * 1.96
    def st_error(upper, lower):
        return  (np.log(upper) - np.log(lower))/3.92
    
    # The weigth of each study is 1/var(estimate) = 1/std-error**2
    def weight(se):
        return 1/(se**2)
    
    # apply the the st-error and weight functions to each row and create new columns containing the new variables.
    df_effect["se"] = df_effect.apply(lambda row : st_error(row["u"], row["l"]), axis = 1)
    df_effect["weight"] = df_effect.apply(lambda row : weight(row["se"]), axis = 1)
    sum_of_weights = df_effect["weight"].sum()
    
    # the pooled effect is calculated by summing the weighted individual log-effects and dividing by the sum of all weights.
    # To get the ratio, we have to take the exp again.
    pooled_effect = np.exp((np.log(df_effect["ratio"])*df_effect["weight"]).sum()/sum_of_weights)
    pooled_se = (1/sum_of_weights)**0.5
    
    # All that is missing now are estimate for the confidence interval bounds. Again we leverage the normality-assumotion for logs and
    # and and substract the product of the standard-errors and the critical z-value.
    pooled_lower_bound = np.exp(np.log(pooled_effect) - 1.96*pooled_se)
    pooled_upper_bound = np.exp(np.log(pooled_effect) + 1.96*pooled_se)
    
    return pooled_effect, pooled_lower_bound, pooled_upper_bound
                                      
    

In [ ]:
# load manually coded result into dataframe
df_results = pd.read_csv("result_table.csv", sep= ";")

# choose subset of columns for mortality-data
df_mortality = df_results[["pmc", "ratio_mortality", "lower_confidence_bound_mortality", "upper_confidence_bound_mortality", 
                          "p_value_mortality", "sample_size_remdesivir_mortality", "sample_size_control_mortality", "placebo_controlled"]]
# drop all rows in which there is no ratio for mortality recorded
df_mortality = df_mortality.dropna(subset=['ratio_mortality'])
df_mortality = df_mortality.reset_index(drop=True)

# do the same for the improvement/recovery data
df_improvement = df_results[["pmc", "ratio_recovery", "lower_confidence_bound_recovery", "upper_confidence_bound_recovery", 
                          "p_value_recovery", "sample_size_remdesivir_recovery", "sample_size_control_recovery", "placebo_controlled"]]
df_improvement = df_improvement.dropna(subset=["ratio_recovery"])
df_improvement = df_improvement.reset_index(drop=True)

# Add rows for pooled effects to data-frames. We create a list of all necessary values for a row. The label, the effect with both confidence bounts, no p-value,
# a sum of all participants per group and no value for placebo-indicator.
row_pooled_mortality = ["mortality_pooled"] + list(pooled_effect(df_mortality))  +[np.nan,df_mortality["sample_size_remdesivir_mortality"].sum(),
                                                                                   df_mortality["sample_size_control_mortality"].sum(),np.nan] 
df_mortality.loc[len(df_mortality)] = row_pooled_mortality
row_pooled_improvement = ["improvement_pooled"] + list(pooled_effect(df_improvement))  +[np.nan,df_improvement["sample_size_remdesivir_recovery"].sum(),
                                                                                         df_improvement["sample_size_control_recovery"].sum(),np.nan] 
df_improvement.loc[len(df_improvement)] = row_pooled_improvement

# change names of columns and reorder them.
df_improvement_processed = pd.DataFrame(columns = ["study", "response","effect-size", "ll", "hl", "p", "n_remdesivir", "n_control", "placebo"])
df_improvement_processed[["study", "effect-size", "ll", "hl", "p", "n_remdesivir", "n_control", "placebo"]] = df_improvement[
    ["pmc", "ratio_recovery", "lower_confidence_bound_recovery", "upper_confidence_bound_recovery", "p_value_recovery", "sample_size_remdesivir_recovery", 
     "sample_size_control_recovery","placebo_controlled"]]

df_improvement_processed["response"] = "recovery"
df_improvement_processed ["placebo"] = df_improvement_processed["placebo"].astype(bool)
df_improvement_processed.loc[df_improvement_processed.study == "improvement_pooled","placebo"] = np.nan

# calculate overall n per study and sort data-frame by n
df_improvement_processed["N"] = df_improvement_processed.n_remdesivir + df_improvement_processed.n_control
df_improvement_processed = df_improvement_processed.sort_values(by=["N"], ascending=False)
df_improvement_processed = df_improvement_processed.reset_index(drop=True)

# for the plots below, each study needs a unique identifier. Some studys are included in both mortality and recovery subgroups. Add an identifier
df_improvement_processed["study"] = df_improvement_processed["study"]+ " r"

# repeat steps for other subgroup
df_mortality_processed = pd.DataFrame(columns = ["study", "response","effect-size", "ll", "hl", "p", "n_remdesivir", "n_control", "placebo"])
df_mortality_processed[["study", "effect-size", "ll", "hl", "p", "n_remdesivir", "n_control", "placebo"]] = df_mortality[
    ["pmc", "ratio_mortality", "lower_confidence_bound_mortality", "upper_confidence_bound_mortality", "p_value_mortality", "sample_size_remdesivir_mortality", "sample_size_control_mortality","placebo_controlled"]]
df_mortality_processed["response"] = "mortality"
df_mortality_processed ["placebo"] = df_mortality_processed["placebo"].astype(bool)
df_mortality_processed.loc[df_mortality_processed.study == "mortality_pooled","placebo"] = np.nan
df_mortality_processed["N"] = df_mortality_processed.n_remdesivir + df_mortality_processed.n_control
df_mortality_processed = df_mortality_processed.sort_values(by=["N"], ascending=False)
df_mortality_processed = df_mortality_processed.reset_index(drop=True)
df_mortality_processed["study"] = df_mortality_processed["study"]+ " m"
df_total = pd.concat([df_mortality_processed,df_improvement_processed] )
df_total = df_total.reset_index(drop=True)
df_total["N"] = df_total["N"].astype("O")
for i in range(len(df_total)):
    if df_total.at[i,"study"] == "improvement_pooled r":
        df_total.at[i,"study"] = "recovery pooled"
    if df_total.at[i,"study"] == "mortality_pooled m":
        df_total.at[i,"study"] = "mortality pooled"
df_total

### Significance Test for pooled effects.
We are going to use the confidence-interval significance test. We are checking if the value 1 is within the 95% Confidence Interval. The value of 1 indicates that the groups do not differ with respect to the outcome-measure,as the measure is a ratio.

In [4]:
mortality_ll = df_total.loc[df_total.study == "mortality pooled","ll"].values[0]
mortality_hl = df_total.loc[df_total.study == "mortality pooled","hl"].values[0]
print("confidence interval for clinical recovery or improvement [{},{}]".format(mortality_ll,mortality_hl))
if mortality_hl < 1:
    print("Treamtment with remdesivir is associated with a significantly lower mortality-rate")
elif mortality_ll > 1:
    print("Treamtment with remdesivir is associated with a significantly higher mortality-rate")
else:
    print("Treamtment with remdesivir is not associated with a significantly faster recovery or clinical improvement")
    
improvement_ll = df_total.loc[df_total.study == "recovery pooled","ll"].values[0]
improvement_hl = df_total.loc[df_total.study == "recovery pooled","hl"].values[0]
print("\nconfidence interval for clinical recovery or improvement [{},{}]".format(improvement_ll,improvement_hl))
if improvement_ll > 1:
    print("Treamtment with remdesivir is associated with a significantly faster recovery or clinical improvement")
elif improvement_hl < 1:
    print("Treamtment with remdesivir is associated with a significantly slower recovery or clinical improvement")
else:
    print("Treamtment with remdesivir is not associated with a significantly faster recovery or clinical improvement")

confidence interval for clinical recovery or improvement [0.7125795963145016,0.9967472701424689]
Treamtment with remdesivir is associated with a significantly lower mortality-rate

confidence interval for clinical recovery or improvement [1.0254684086906964,1.2280504410758566]
Treamtment with remdesivir is associated with a significantly faster recovery or clinical improvement


### Create Forestplot
Now that we know if the pooled outcome was significant or not, let's visualize our results with a forest plot.

In [ ]:
fp.forestplot(df_total,  # the dataframe with results data
              estimate="effect-size",  # col containing estimated effect size 
              ll="ll", hl="hl",  # lower & higher limits of conf. int.
              varlabel="study",  # column containing the varlabels to be printed on far left
              pval="p",  # column containing p-values to be formatted
              annote=["N"],  # columns to report on left of plot
              annoteheaders=["N"],  # ^corresponding headers
              rightannote=["formatted_pval", "est_ci", "placebo"],  # columns to report on right of plot 
              right_annoteheaders=["p","Confidence Interval","Placebo"],  # ^corresponding headers
              groupvar = "response",
              group_order=["mortality", "recovery"],    
              xlabel="ratio",  # x-label title
              xline = 1,
              xlinestyle = (1, (10, 5)),
              figsize = (8,8),
            table=True)
plt.suptitle('Forest Plot of Rate- or Hazard-Ratios for Effect of Remdesivir on Covid-19', fontsize=16)
plt.figtext(-0.2, 0.01, 'Note: The Variable-column lists the pubmed-central-id of the study for open access. \"m\" stands for mortality, and \"r\" for recovery', ha="left", fontsize=12)
plt.savefig('Forest Plot.png', bbox_inches='tight')
plt.show()

### Create Funnelplots
We can test for publication-bias graphically with funnelplots. Funnelplots show the expected variations of effects, given that there is no publication bias. The larger the standard-error of an effect is, the larger the expected variability in reported effects. If there is no publication-bias, the effects will be symmetrically aligned around the line of the pooled effect-size. If there is a publication bias, the studies with smaller effect will be skewed towards the significant side. In the case of the effect size being mortality, the effect of studies with smaller sample sizes (larger standard error) will be skewed towards the left of the plot, while for the recovery-measure they will be skewed towards the right of the plot. This however only meaningfully works, if there are a sufficient number of studies, which might not be the case <br>
The effect-sizes only vary symmetrically if we take the logs of the rates first. The lined diagonal blue lines show where 95% of effects is expected to be in for the pooled effect-size.

In [ ]:
# Run the cell to create funnel plot for mortality
df_mortality_processed["SE"] = (np.log(df_mortality_processed.hl) - np.log(df_mortality_processed.ll))/3.92
fig=plt.figure(figsize=(9,8))
fig.suptitle('Funnel Plot of Logarithmic Mortality Hazard-Ratios', fontsize=16)
ax=plt.gca()
x=np.log(df_mortality_processed.loc[1:,"effect-size"])
y=df_mortality_processed["SE"].loc[1:]
log_effect_pooled=np.log(df_mortality_processed.loc[0, "effect-size"])   

plt.xlim(min(x)-1.1*1.96*max(y),max(x)+1.1*1.96*max(y))  #the xlim can change manually
plt.ylim(max(y)+0.02,0)

plt.plot ([0,(-1.96*max(y))],[0,max(y)], linestyle="None", lw=1)
plt.plot ([0,(1.96*max(y))],[0,max(y)], linestyle="None", lw=1)

plt.plot ([log_effect_pooled,log_effect_pooled],[0,max(y)],color="blue", linestyle="--", lw=1)
plt.plot ([log_effect_pooled,log_effect_pooled-1.96*max(y)],[0,max(y)],color="blue", linestyle="--", lw=1)
plt.plot ([log_effect_pooled,log_effect_pooled+1.96*max(y)],[0,max(y)],color="blue", linestyle="--", lw=1)
plt.plot(x,y,"o",lw=1,color="k")

plt.xlabel("Log Hazard-Ratio",fontsize=14 )
plt.ylabel("Standard Error",fontsize=14,rotation=90 )

plt.show()

In [ ]:
# Run the cell to create funnel plot for recovery
from matplotlib.lines import Line2D
df_improvement_processed["SE"] = (np.log(df_improvement_processed.hl) - np.log(df_improvement_processed.ll))/3.92
fig=plt.figure(figsize=(9,8))
fig.suptitle('Funnel Plot of Logarithmic Recovery Rate-Ratios', fontsize=16)
ax=plt.gca()
x=np.log(df_improvement_processed.loc[1:,"effect-size"])
y=df_improvement_processed.loc[1:,"SE"]     
log_effect_pooled=np.log(df_improvement_processed.loc[0, "effect-size"])   

plt.xlim(min(x)-1.1*1.96*max(y),max(x)+1.1*1.96*max(y))  #the xlim can change manually
plt.ylim(max(y)+0.02,0)
   

plt.plot ([0,(-1.96*max(y))],[0,max(y)], linestyle="None", lw=1)
plt.plot ([0,(1.96*max(y))],[0,max(y)], linestyle="None", lw=1)

plt.plot ([log_effect_pooled,log_effect_pooled],[0,max(y)],color="blue", linestyle="--", lw=1)
plt.plot ([log_effect_pooled,log_effect_pooled-1.96*max(y)],[0,max(y)],color="blue", linestyle="--", lw=1)
plt.plot ([log_effect_pooled,log_effect_pooled+1.96*max(y)],[0,max(y)],color="blue", linestyle="--", lw=1)
plt.plot(x,y,"o",lw=1,color="k")

plt.xlabel("Log Rate-Ratio",fontsize=14 )
plt.ylabel("Standard Error",fontsize=14,rotation=90 )

plt.show()

Congratulations, you finished the systematic review. Now you can compare your results to mine. My plots can be found in the zip-folder results_locked. The password is Ilovesystematicreview